# 3. Cleaning the dataset

Before we can construct our network we cleaned our dataset. There are different levels in our cleaning process
* Remove duplicates
* Remove spam emails
* Remove emails from infrequent email addresses
* Group all not @enron emails together into a specific domain, one for sending and one for receiving
* Set a threshold for how many emails between 2 parties must have been exchanged before being taken into account

In [2]:
# Import packages
import pandas as pd
import numpy as np

## 3.1 Load in data frame

In [4]:
# Direct mails
Direct_mails = pd.read_csv('All direct mails')
print(Direct_mails.shape)
Direct_mails.head()

(900893, 5)


,From,To,Date,Subject,Message_ID
0,pallen70@hotmail.com,pallen@enron.com,2001-12-30 10:19:42,Fwd: Bishops Corner,215433951075855374340
1,arsystem@mailman.enron.com,k..allen@enron.com,2001-12-27 17:16:46,Your Approval is Overdue: Access Request for m...,253634511075855374674
2,webmaster@earnings.com,pallen@enron.com,2001-10-18 15:21:22,NT Earnings Information,197054941075858631723
3,discount@open2win.oi3.net,pallen@enron.com,2001-10-18 14:57:30,50% Hotel Discount Notice #7734228 for PHILLIP,79963351075858632216
4,no.address@enron.com,All Enron Worldwide@ENRON,2001-10-18 15:10:12,UPDATE - Supported Internet Email Addresses,89140651075858632242


In [47]:
# Check for duplicates
Direct_mails.duplicated().value_counts()

False    894024
True       6869
dtype: int64

In [48]:
# Remove duplicated values
Direct_mails = Direct_mails[~Direct_mails.duplicated()]

In [49]:
Direct_mails.shape

(894024, 5)

## 3.2 Remove spam

For our project we are interested in looking at the emails between employees. In this section we filter out the emails which are related to spam.

**1. Create spam detector**

First of all we will remove the spam emails from the dataset. We start by creating a spam detector. There is a subset of the Enron email corpus which is labeled whether an email is spam or ham. We try to create a classifier which is able to classify an email into spam or ham based on the subject. We achieved an accuracy of 0.94 with tf-idf scores and a support vector machine. This is good enough for our project.

In [50]:
# Load in dataset
spam_data = pd.read_csv('enron_spam_data.csv')

In [51]:
# Check output
spam_data.head()

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14


In [52]:
# Rename column
spam_data = spam_data.rename(columns={'Spam/Ham': 'Category'})

In [53]:
# Remove empty values in the subject column
spam_data = spam_data[~spam_data.Subject.isnull()]

In [54]:
# Install packages
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
# Split the data
train, test = train_test_split(spam_data[['Subject', 'Category']])

In [56]:
# Check distribution
train.Category.value_counts()

spam    12656
ham     12414
Name: Category, dtype: int64

In [57]:
# Check distribution
test.Category.value_counts()

spam    4226
ham     4131
Name: Category, dtype: int64

In [58]:
# Create TFIDF vectorizer
tf_idf = TfidfVectorizer()
# For the train set
X_train = tf_idf.fit_transform(train.Subject)
print(X_train.shape)
# For the test set
X_test = tf_idf.transform(test.Subject)
print(X_test.shape)

(25070, 15401)
(8357, 15401)


In [59]:
# Install packages
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [60]:
# Create model
clf = svm.SVC()

In [61]:
# Train model
clf.fit(X_train, train.Category)

SVC()

In [62]:
# Check training accuracy
clf.score(X_train, train.Category)

0.9878340646190666

In [63]:
# Test accuracy
clf.score(X_test, test.Category)

0.9445973435443341

In [64]:
# Store predictions
pred_svm = clf.predict(X_test)

In [65]:
# Check output
print(classification_report(test.Category, pred_svm))

              precision    recall  f1-score   support

         ham       0.95      0.94      0.94      4131
        spam       0.94      0.95      0.95      4226

    accuracy                           0.94      8357
   macro avg       0.94      0.94      0.94      8357
weighted avg       0.94      0.94      0.94      8357



**2. Preprocess the subject of all emails.**

Now that we have created a classifier to detect emails related to spam we can use this classifier on the full Enron data set. We start with filtering out the the emails who did not have a subject. Next we go through some text preprocessing steps.

In [66]:
# Get all subject from direct mails
Direct_subject = Direct_mails[~Direct_mails.Subject.isnull()].Subject

In [67]:
Direct_subject[:10]

0                                  Fwd: Bishops Corner
1    Your Approval is Overdue: Access Request for m...
2                              NT Earnings Information
3       50% Hotel Discount Notice #7734228 for PHILLIP
4          UPDATE - Supported Internet Email Addresses
5                Conference Call Today with FERC Staff
6                Conference Call Today with FERC Staff
7                Conference Call Today with FERC Staff
8                Conference Call Today with FERC Staff
9                Conference Call Today with FERC Staff
Name: Subject, dtype: object

In [68]:
# Set to lower case
Preproces_subject = Direct_subject.map(lambda x: x.lower())

In [69]:
# Import package
import re

In [70]:
# Remove punctuations
Preproces_subject = Preproces_subject.map(lambda x: re.sub(r'[^\w\s]', '', x))

In [71]:
# Get stopwords
from nltk.corpus import stopwords

In [72]:
# Save english stopwords
StopWords = stopwords.words("english")

# Create function to remove stopwords
def RemoveStopwords(text):
    text_nostopwords = ' '.join([word for word in text.split() if word not in StopWords])
    return text_nostopwords

In [73]:
# Remove stopwords
Preproces_subject = Preproces_subject.map(lambda x: RemoveStopwords(x))

In [74]:
Preproces_subject[:10]

0                                   fwd bishops corner
1    approval overdue access request mattsmithenroncom
2                              nt earnings information
3             50 hotel discount notice 7734228 phillip
4            update supported internet email addresses
5                     conference call today ferc staff
6                     conference call today ferc staff
7                     conference call today ferc staff
8                     conference call today ferc staff
9                     conference call today ferc staff
Name: Subject, dtype: object

In [75]:
# Use TFIDF vectorizer
TFIDF_Enron = tf_idf.transform(Preproces_subject)
TFIDF_Enron.shape

(878276, 15401)

In [76]:
# Use classifier on all enron mails
pred_enron = clf.predict(TFIDF_Enron)

In [77]:
# Check the distribution of spam vs ham mails
pd.Series(pred_enron).value_counts()

ham     713522
spam    164754
dtype: int64

In [78]:
# Check some examples of spam mail
Direct_subject[pred_enron=='spam'][:10]

3        50% Hotel Discount Notice #7734228 for PHILLIP
64                     Meet the dark side of Windows XP
66    Apply online for a No Deposit VISA or Master C...
73                   E-delivery Notification - Confirms
74          Your Weekly Movie Showtimes from Amazon.com
81                                              workout
82                  Credit Watch List--Week of 10/22/01
83                  Credit Watch List--Week of 10/22/01
84                  Credit Watch List--Week of 10/22/01
85                  Credit Watch List--Week of 10/22/01
Name: Subject, dtype: object

It looks like the classifier overestimated the amount of spam emails in the dataset. To resolve this issue we will filter out the email adresses who ended on "@enron.com" as these are definitely not related to spam.

In [79]:
# Create dataframe storing the subject and corresponding category
Spam_mail = pd.DataFrame({'Subject':Direct_subject, 'Category':pred_enron})

In [80]:
Spam_mail[:10]

,Subject,Category
0,Fwd: Bishops Corner,ham
1,Your Approval is Overdue: Access Request for m...,ham
2,NT Earnings Information,ham
3,50% Hotel Discount Notice #7734228 for PHILLIP,spam
4,UPDATE - Supported Internet Email Addresses,ham
5,Conference Call Today with FERC Staff,ham
6,Conference Call Today with FERC Staff,ham
7,Conference Call Today with FERC Staff,ham
8,Conference Call Today with FERC Staff,ham
9,Conference Call Today with FERC Staff,ham


In [81]:
# Concatenate the two dataframe
Direct_mails_spam = pd.concat([Direct_mails, Spam_mail.Category], axis=1)

In [82]:
# Groupby from and category to see which email adresses were classified as spam
grouped = Direct_mails_spam.groupby(['Category', 'From']).count()

In [83]:
grouped.head()

To  Date  Subject  Message_ID
Category From                                                              
ham      'todd'.delahoussaye@enron.com         1     1        1           1
         --migrated--bmishkin@ercot.com        1     1        1           1
         --migrated--dodle@ercot.com           1     1        1           1
         -persson@ricemail.ricefinancial.com   1     1        1           1
         01@ftenergy.com                       4     4        4           4

In [84]:
# Extract all adressess classified as spam
possible_spam_adresses = grouped.loc['spam'].index.tolist()

In [85]:
# Filter out the adressses ending on enron.com
spam_adresses = [adress for adress in possible_spam_adresses if not bool(re.search(r'.enron\.com$', adress))]

In [86]:
# Check length
len(spam_adresses)

4466

In [87]:
# Filter from main dataset
Direct_mails = Direct_mails[~Direct_mails.From.isin(spam_adresses)]

In [88]:
# Check shape
Direct_mails.shape

(834471, 5)

In total 59.353 emails were removed from our dataset.

## 3.3 Remove infrequent email addresses

In [89]:
# Store the total amount of send and received emails for each email address
Count_From = Direct_mails.From.value_counts()
Count_To = Direct_mails.To.value_counts()

In [90]:
# Sum them together
Total_Send_Received = pd.concat([Count_From, Count_To], axis=1).sum(axis=1).sort_values(ascending=False)

In [91]:
# Get addresses of interest
AdressesofInterest = Total_Send_Received[Total_Send_Received>=10].index.tolist()

In [92]:
# filter based upon these addresses
Direct_mails = Direct_mails[Direct_mails.From.isin(AdressesofInterest) & Direct_mails.To.isin(AdressesofInterest)]

In [93]:
# Check shape
Direct_mails.shape

(779139, 5)

We removed another 55.332 emails from the dataset.

## 3.4 Group external email domains

For this project we are interested in communication between employees, we will be focussing on the emails ending on @enron.com although there might still be a lot of information in the email addresses with a different domain. We grouped all email addresses with the same domain together. When the email domain was rare (<100) the email address was labeled external.

In [94]:
# Count all emails per domain
Count_domains_from = Direct_mails.From.map(lambda x: re.split(r'@', x)).map(lambda x: x[-1]).value_counts()
Count_domains_to = Direct_mails.To.astype(str).map(lambda x: re.split(r'@', x)).map(lambda x: x[-1]).value_counts()

In [95]:
# Sum send and received for every email domain
Domain_send_received = pd.concat([Count_domains_from, Count_domains_to], axis=1).sum(axis=1).sort_values(ascending=False)

In [96]:
# Get all the domains which occur fewer than 100 times
RareDomains = Domain_send_received[Domain_send_received<100].index.tolist()

In [97]:
# Replace the separate rare domain names with External both for sender and receiver
Domains_From = Direct_mails.From.map(lambda x: re.split(r'@', x)).map(lambda x: x[-1]).replace(RareDomains, 'External')
Domains_To = Direct_mails.To.astype(str).map(lambda x: re.split(r'@', x)).map(lambda x: x[-1]).replace(RareDomains, 'External')

In [98]:
# Add from so we can differentiate between send and received
Domains_From = Domains_From.map(lambda x: 'From_' + x)
Domains_To = Domains_To.map(lambda x: 'To_' + x)

In [99]:
Domains_From.head()

0          From_hotmail.com
1    From_mailman.enron.com
4            From_enron.com
5            From_enron.com
6            From_enron.com
Name: From, dtype: object

In [100]:
Domains_To.head()

0    To_enron.com
1    To_enron.com
4     To_External
5    To_enron.com
6    To_enron.com
Name: To, dtype: object

In [101]:
# Replace the non enron email addresses with their respective domain name
Direct_mails.loc[Domains_From[~Domains_From.isin(['From_enron.com', 'From_enron.com>'])].index,'From'] = Domains_From[~Domains_From.isin(['From_enron.com', 'From_enron.com>'])]
Direct_mails.loc[Domains_To[~Domains_To.isin(['To_enron.com', 'To_enron.com>'])].index,'To'] = Domains_To[~Domains_To.isin(['To_enron.com', 'To_enron.com>'])]

## 3.5 Remove interactions that only happened once

We won't incorporate interaction in our network that only occured 1 once. 

In [104]:
# Look at different interactions
Direct_mails.groupby(['From', 'To']).Message_ID.count()

From                           To                           
'todd'.delahoussaye@enron.com  'todd'.delahoussaye@enron.com    5
                               anne.bike@enron.com              1
                               bianca.ornelas@enron.com         5
                               brant.reves@enron.com            5
                               c..gossett@enron.com             5
                                                               ..
zulie.flores@enron.com         sharon.butcher@enron.com         3
                               sheila.walton@enron.com          3
                               terrie.james@enron.com           3
                               terrie.wheeler@enron.com         3
                               tim.o'rourke@enron.com           3
Name: Message_ID, Length: 91434, dtype: int64

We need to create an additional variable which specifies the interaction between employees and doesn't consider who sended and who received the message. To achieve this goal we will join from and to together in a single variable and sort the names in alphabetical order.

In [105]:
# Create defintion which sort the names in alphabetical order
def SortAlphabetical(string):
    # Split the string
    string = string.split('-')
    # Sort the values alphabetically
    string.sort()
    # join back together
    string = '-'.join(string)
    # Return string
    return string

In [106]:
# Create variable showing the interaction
Direct_mails['Interaction'] = Direct_mails[['From', 'To']].apply('|'.join, axis=1).map(lambda x: SortAlphabetical(x))

In [108]:
# Check how often all interactions occur
Count_Interactions = Direct_mails.Interaction.value_counts()
Count_Interactions

pete.davis@enron.com|pete.davis@enron.com              2413
vince.kaminski@enron.com|To_aol.com                    2206
kay.mann@enron.com|suzanne.adams@enron.com              870
jeff.dasovich@enron.com|To_External                     865
kay.mann@enron.com|To_kslaw.com                         773
                                                       ... 
ted.murphy@enron.com|mark.frevert@enron.com               1
bradford.larson@enron.com|mark.bonney@enron.com           1
bradford.larson@enron.com|bjorn.hagelmann@enron.com       1
bradford.larson@enron.com|don.rollins@enron.com           1
paul.y'barbo@enron.com|david.shields@enron.com            1
Name: Interaction, Length: 91434, dtype: int64

In [111]:
Direct_mails.Interaction.value_counts().map(lambda x: x>=2).value_counts(normalize=True)

True     0.628924
False    0.371076
Name: Interaction, dtype: float64

In [112]:
# define a threshold and select a subset of interactions
Interactions = Count_Interactions[Count_Interactions>1].index.tolist()

In [114]:
# Take a subset of our dataframe
Direct_mails = Direct_mails[Direct_mails.Interaction.isin(Interactions)]

In [115]:
# Check output
print(Direct_mails.shape)
Direct_mails.head()

(745210, 6)


,From,To,Date,Subject,Message_ID,Interaction
0,From_hotmail.com,pallen@enron.com,2001-12-30 10:19:42,Fwd: Bishops Corner,215433951075855374340,From_hotmail.com|pallen@enron.com
1,From_mailman.enron.com,k..allen@enron.com,2001-12-27 17:16:46,Your Approval is Overdue: Access Request for m...,253634511075855374674,From_mailman.enron.com|k..allen@enron.com
4,no.address@enron.com,To_External,2001-10-18 15:10:12,UPDATE - Supported Internet Email Addresses,89140651075858632242,no.address@enron.com|To_External
5,ray.alvarez@enron.com,j..kean@enron.com,2001-10-18 14:51:19,Conference Call Today with FERC Staff,317060761075858632278,ray.alvarez@enron.com|j..kean@enron.com
6,ray.alvarez@enron.com,richard.shapiro@enron.com,2001-10-18 14:51:19,Conference Call Today with FERC Staff,317060761075858632278,ray.alvarez@enron.com|richard.shapiro@enron.com


## 3.6 Check result

In [152]:
Direct_mails.head()

,From,To,Date,Subject,Message_ID,Interaction
0,From_hotmail.com,pallen@enron.com,2001-12-30 10:19:42,Fwd: Bishops Corner,215433951075855374340,From_hotmail.com|pallen@enron.com
1,From_mailman.enron.com,k..allen@enron.com,2001-12-27 17:16:46,Your Approval is Overdue: Access Request for m...,253634511075855374674,From_mailman.enron.com|k..allen@enron.com
4,no.address@enron.com,To_External,2001-10-18 15:10:12,UPDATE - Supported Internet Email Addresses,89140651075858632242,no.address@enron.com|To_External
5,ray.alvarez@enron.com,j..kean@enron.com,2001-10-18 14:51:19,Conference Call Today with FERC Staff,317060761075858632278,ray.alvarez@enron.com|j..kean@enron.com
6,ray.alvarez@enron.com,richard.shapiro@enron.com,2001-10-18 14:51:19,Conference Call Today with FERC Staff,317060761075858632278,ray.alvarez@enron.com|richard.shapiro@enron.com


In [153]:
# Check shape
Direct_mails.shape

(745210, 6)

In [154]:
# export
Direct_mails.to_csv('files/DirectMails_clean.csv', index = False)